<a href="https://colab.research.google.com/github/schatz06/EPP445_Project/blob/main/eye_landamarks_recognition_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages

In [2]:
import cv2
import dlib
import os
import re

Regex

In [13]:
REG_PART = re.compile("part name='[0-9]+'")
REG_NUM = re.compile("[0-9]+")

Here we define the landmark point for the eyes.

*   68 point in a face
*   22 points for eyes and eyebrows






In [3]:
EYE_EYEBROWS = [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 36, 37, 38, 39,
                40, 41, 42, 43, 44, 45, 46, 47]

Dataset used iBug 300W


*   Download
*   Extract






In [ ]:
!wget http://dlib.net/files/data/ibug_300W_large_face_landmark_dataset.tar.gz

In [ ]:
!tar xvzf ibug_300W_large_face_landmark_dataset.tar.gz

Paths 

In [14]:
ibug_dir = "ibug_300W_large_face_landmark_dataset"
train_labels = "labels_ibug_300W_train.xml"
test_labels = "labels_ibug_300W_test.xml"